# Connectome Manipulation : Connection Removal

## Introduction

Welcome to this demonstration on manipulating connectomes within SONATA circuit models. This Jupyter notebook serves as a hands-on guide to understanding and implementing techniques for the modification of neural circuitry, with a particular focus on the removal of connections. Our exploration is designed to equip researchers and practitioners with the necessary tools and knowledge to advance their work in computational neuroscience and related fields.

We will be removing 20% of connections between excitatory neurons in the circuit and 30% of the Inhibitory to Excitatory connections.

### Step 1: Download example circuit

Download and uncompress SONATA network model of the rat somatosensory cortex from ZENODO (DOI [10.5281/zenodo.7930275](https://zenodo.org/doi/10.5281/zenodo.7930275)).


### Step 2: Load necessary libraries and investigate the circuit content


In [1]:
""" Global imports """
import json
import os
from pprint import pprint

""" Local imports """

from connectome_manipulator.connectome_manipulation.manipulation import Manipulation
# from connectome_manipulator.connectome_manipulation import connectome_manipulation
# from connectome_manipulator.connectome_comparison import structural_comparator
from pathlib import Path

In [2]:
#import connectome_manipulator

In [11]:
# import circuit
circuit_path = '/gpfs/bbp.cscs.ch/project/proj83/jira-tickets/NSETM-1948-extract-hex-O1/data/O1_data_physiology' # adjust the circuit config path
circuit_config_path = f'{circuit_path}/circuit_config.json'
circuit_name = 'SSCx'
node_set = 'hex0'

working_dir = './'
output_path = './output'
os.makedirs(output_path,exist_ok=True)

In [12]:
from bluepysnap import Circuit

c = Circuit(circuit_config_path)

In [13]:
c.nodes._populations

{'POm': <bluepysnap.nodes.node_population.NodePopulation at 0x2aab7e950520>,
 'S1nonbarrel_neurons': <bluepysnap.nodes.node_population.NodePopulation at 0x2aab7e950f10>,
 'VPM': <bluepysnap.nodes.node_population.NodePopulation at 0x2aab7e950fd0>,
 'external_S1nonbarrel_neurons__S1nonbarrel_neurons__chemical': <bluepysnap.nodes.node_population.NodePopulation at 0x2aab7e950d60>,
 'external_midrange__S1nonbarrel_neurons__chemical': <bluepysnap.nodes.node_population.NodePopulation at 0x2aab7e95e040>}

In [14]:
c.edges._populations

{'POm__S1nonbarrel_neurons__chemical': <bluepysnap.edges.edge_population.EdgePopulation at 0x2aab7e95e0d0>,
 'S1nonbarrel_neurons__S1nonbarrel_neurons__chemical': <bluepysnap.edges.edge_population.EdgePopulation at 0x2aab7e95e190>,
 'VPM__S1nonbarrel_neurons__chemical': <bluepysnap.edges.edge_population.EdgePopulation at 0x2aab7e95e220>,
 'external_S1nonbarrel_neurons__S1nonbarrel_neurons__chemical': <bluepysnap.edges.edge_population.EdgePopulation at 0x2aab7e95e2b0>,
 'external_midrange__S1nonbarrel_neurons__chemical': <bluepysnap.edges.edge_population.EdgePopulation at 0x2aab7e95e280>,
 'midrange__S1nonbarrel_neurons__chemical': <bluepysnap.edges.edge_population.EdgePopulation at 0x2aab7e95e3a0>}

In [15]:
pprint(c.config)

{'components': {'biophysical_neuron_models_dir': '/gpfs/bbp.cscs.ch/project/proj83/jira-tickets/NSETM-1948-extract-hex-O1/data/O1_data_physiology',
                'mechanisms_dir': '/gpfs/bbp.cscs.ch/project/proj83/jira-tickets/NSETM-1948-extract-hex-O1/data/O1_data_physiology',
                'morphologies_dir': '/gpfs/bbp.cscs.ch/project/proj83/jira-tickets/NSETM-1948-extract-hex-O1/data/O1_data_physiology',
                'point_neuron_models_dir': '',
                'provenance': {'atlas_dir': '/gpfs/bbp.cscs.ch/project/proj83/jira-tickets/NSETM-1948-extract-hex-O1/data/O1_data_physiology/atlas'},
                'synaptic_models_dir': '',
                'templates_dir': ''},
 'networks': {'edges': [{'edges_file': '/gpfs/bbp.cscs.ch/project/proj83/jira-tickets/NSETM-1948-extract-hex-O1/data/O1_data_physiology/S1nonbarrel_neurons__S1nonbarrel_neurons__chemical/edges.h5',
                         'populations': {'S1nonbarrel_neurons__S1nonbarrel_neurons__chemical': {'type': 'che

### Step 3: Create the Manipulation Config

Now that we know our node and edge populations in the circuit, we can start adressing them in the config file to specify what type of manipulation we want to perform

In [34]:
# Manipulation specifications
manip_config = {}

amount_pct = 20 # Amount of target neurons to rewire afferent connections to
edge_pop_name= 'S1nonbarrel_neurons__S1nonbarrel_neurons__chemical'

manip_config['edges_popul_name'] = edge_pop_name
manip_config['manip'] = {'name': f'{circuit_name}_remove_conns',
                         'fcts': [
                            {'source': 'conn_removal',
                                   'sel_src': {'node_set': node_set,'synapse_class': 'EXC'},
                                    'sel_dest': {'node_set': node_set},
                                    'amount_pct': amount_pct,
                                    'model_config':{}
                            }
                            ]}

## General settings
manip_config['circuit_config'] = f"./{os.path.relpath(circuit_config_path, circuit_path)}" # SONATA (.json) format; path rel. to 'circuit_path'
manip_config['seed'] = 3220

In [22]:
pprint(manip_config)

{'circuit_config': './circuit_config.json',
 'edges_popul_name': 'S1nonbarrel_neurons__S1nonbarrel_neurons__chemical',
 'manip': {'fcts': [{'amount_pct': 20,
                     'model_config': {},
                     'sel_dest': {'node_set': 'hex0'},
                     'sel_src': {'node_set': 'hex0', 'synapse_class': 'EXC'},
                     'source': 'conn_removal'}],
           'name': 'SSCx_remove_conns'},
 'seed': 3220}


In [24]:
# Export model config for sbatch run

with open(os.path.join(working_dir, f'manip_config.json'), 'w') as f:
    json.dump(manip_config, f, indent=2)

### Step 3: Run the connectome manipulator

Now that we defined the manipulator config file, we can call the main method of the connectome manipulator from python. 

~~~
sbatch run_rewiring_parallel.sh "manip_config.json" "/path/to/output" 500
~~~

### Step 4: Examine the manipulated circuit

You can then use structural comparator feature of connectome manipulator to see effect of manipulation on the circuit

In [26]:
structural_comparison_output_path = f'{output_path}/structcomp'

In [27]:
os.makedirs

'./output/structcomp'

In [36]:
with open('structcomp_config__Orig_vs_Manip.json', 'r') as f:
    cfg_dict = json.load(f)
cfg_dict['working_dir'] = structural_comparison_output_path
cfg_dict['out_dir'] = structural_comparison_output_path
cfg_dict['circuits']['0']['circuit_config'] = circuit_config_path
cfg_dict['circuits']['1']['circuit_config'] = f'{output_path}/circuit_config.json'

# add edges_popul_name for each analysis

for plot_type in cfg_dict['plot_types']:
    plot_type['fct']['kwargs']['edges_popul_name'] = edge_pop_name

with open(f'{working_dir}/structcomp_config.json', 'w') as f:
    json.dump(cfg_dict, f, indent=2)

In [33]:
cfg_dict.keys()

dict_keys(['working_dir', 'out_dir', 'circuits', 'plot_types'])